In [ ]:
import torch
import numpy as np
from torch.autograd import Variable

from models.cond_pulse2pulse import CondP2PDiscriminator
from models.cond_pulse2pulse import CondP2PGenerator

In [ ]:
chk_path = "checkpoint_epoch:3000.pt"

netD = WaveGANDiscriminator(model_size=50, ngpus=1).cuda()
netG = WaveGANGenerator(model_size=50, ngpus=1).cuda()

chkpnt = torch.load(chk_path)

netD.load_state_dict(chkpnt["netD_state_dict"])
netG.load_state_dict(chkpnt["netG_state_dict"])

In [ ]:
x = Variable(torch.randn(32, 8, 1000)).cuda()

In [ ]:
labels = np.load('test_labels.npy')[0:32]
labels = torch.from_numpy(labels).cuda().float()

In [ ]:
out = netG(x, labels)

In [ ]:
def generate_four_leads(tensor):
    leadI = tensor[:,0,:].unsqueeze(1)
    leadschest = tensor[:,1:7,:]
    leadavf = tensor[:,7,:].unsqueeze(1)


    leadII = (0.5*leadI) + leadavf

    leadIII = -(0.5*leadI) + leadavf
    leadavr = -(0.75*leadI) -(0.5*leadavf)
    leadavl = (0.75*leadI) - (0.5*leadavf)


    leads12 = torch.cat([leadI, leadII, leadschest, leadIII, leadavr, leadavl, leadavf], dim=1)

    return leads12

In [ ]:
imputations12 = generate_four_leads(out)